In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="2944a580-6c5f-4258-8c86-b9c5de957998"
RESOURCE_GROUP="calebebraga08-rg"
WS_NAME="tcc-experiments"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
# Verify that the handle works correctly.  
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

eastus2 : calebebraga08-rg


In [3]:
TRAIN_DATAS = [
    "vrex_1999_2000_2001_2002_2003_.csv",
    "vrex_2004_2005_2006_2007_2008_.csv",
    "vrex_2009_2010_2011_2012_2013_.csv",
    "vrex_2014_2015_2016_2017_2018_.csv",
]

TEST_DATAS = [
    "vrex_2004.csv",
    "vrex_2009.csv",
    "vrex_2014.csv",
    "vrex_2019.csv"
]

version = "original"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/bb48172e3ccf65075606876bcf5df9f2/vrex_1999_2000_2001_2002_2003_.csv - name: vrex_1999_2000_2001_2002_2003_
Data to test asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/3b205fc48a24468cd8575de6f0de7ae0/vrex_2004.csv - name: vrex_2004
Data to train asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/992a66ec6cfe9db64c4d0613ce5b5208/vrex_2004_2005_2006_2007_2008_.csv - name: vrex_2004_2005_2006_2007_2008_
Data to test asset URI: azureml://subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourcegroups/calebebraga08-rg/workspaces/tcc-experiments

# Load enviroment depedencies

In [4]:
from azure.ai.ml.entities import Environment

custom_env_name = "data-prep-dependencies"

data_prep_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for data preparation",
    tags={"scikit-learn": "0.24.2"},
    conda_file="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/dependencies/conda_data_prep.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="experimental",
)
data_prep_job_env = ml_client.environments.create_or_update(data_prep_job_env)

print(
    f"Environment with name {data_prep_job_env.name} is registered to workspace, the environment version is {data_prep_job_env.version}"
)

Environment with name data-prep-dependencies is registered to workspace, the environment version is experimental


In [5]:
from azure.ai.ml.entities import Environment

custom_env_name = "train-dependencies"

train_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for train data",
    tags={"scikit-learn": "0.24.2"},
    conda_file="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/dependencies/conda_train.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="experimental",
)
train_job_env = ml_client.environments.create_or_update(train_job_env)

print(
    f"Environment with name {train_job_env.name} is registered to workspace, the environment version is {train_job_env.version}"
)

Environment with name train-dependencies is registered to workspace, the environment version is experimental


# Load components

In [6]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
data_prep_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/components/definitions/data_prep.yaml")

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Component data_prep_vrex_defaults_model with Version 2024-03-25-19-56-41-2384834 is registered


In [7]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/components/definitions/train.yaml")

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Component train_vrex_defaults_model with Version 2024-03-25-19-56-43-0621284 is registered


# Pipeline

In [8]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def prep_data_and_train_defaults_pipeline(
    data_to_train,
    data_to_test,
    learning_rate_to_train,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
    )

    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=learning_rate_to_train,  # note: using a pipeline input as parameter
    )


In [9]:
pipelines = []

for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):
    pipeline = prep_data_and_train_defaults_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train),
        data_to_test=Input(type="uri_file", path=data_to_test),
        learning_rate_to_train=0.05
    )
    
    pipelines.append(pipeline)

In [10]:
data_prep_component

CommandComponent({'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'data_prep_vrex_defaults_model', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/2944a580-6c5f-4258-8c86-b9c5de957998/resourceGroups/calebebraga08-rg/providers/Microsoft.MachineLearningServices/workspaces/tcc-experiments/components/data_prep_vrex_defaults_model/versions/2024-03-25-19-56-41-2384834', 'Resource__source_path': None, 'base_path': PosixPath('.'), 'creation_context': <azure.ai.ml._restclient.v2022_10_01.models._models_py3.SystemData object at 0x7ff94971abc0>, 'serialize': <msrest.serialization.Serializer object at 0x7ff94971b1f0>, 'command': 'python train.py  --data_to_train ${{inputs.data_to_train}}  --data_to_test ${{inputs.data_to_test}}  --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}}', 'code': '/subscriptions/2944a580-6c

In [11]:
experiment_names = []
for train_name, test_name in zip(TRAIN_DATAS, TEST_DATAS):
    name = f"{train_name.split('.')[0]}tested_{test_name.split('.')[0]}"
    experiment_names.append(name)
    print(name)



for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )

    ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


vrex_1999_2000_2001_2002_2003_tested_vrex_2004
vrex_2004_2005_2006_2007_2008_tested_vrex_2009
vrex_2009_2010_2011_2012_2013_tested_vrex_2014
vrex_2014_2015_2016_2017_2018_tested_vrex_2019


HttpResponseError: (UserError) Cannot find port definition for port data_to_train from module 65594ef7-3a42-4772-bf78-d4406a0efbce, which is referenced by module node 1f0e87e6 and marked as destination of dataset node a79946a02cd4720095d012b46480a199 with Aml Dataset id 
Code: UserError
Message: Cannot find port definition for port data_to_train from module 65594ef7-3a42-4772-bf78-d4406a0efbce, which is referenced by module node 1f0e87e6 and marked as destination of dataset node a79946a02cd4720095d012b46480a199 with Aml Dataset id 
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "ba1194ae3b2ba8a54f48018e21eb34c0",
        "request": "f92ca705858c0c7a"
    }
}Type: Environment
Info: {
    "value": "eastus2"
}Type: Location
Info: {
    "value": "eastus2"
}Type: Time
Info: {
    "value": "2024-03-25T19:56:46.2075099+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgumen